Demostración SQLite

In [17]:
import sqlite3

def stablish_connection(database):
    conn = sqlite3.connect(database=database)
    return conn

def close_connection(connection):
    connection.close()

def create_cursor(conn):
    return conn.cursor()

def close_cursor(cursor):
    cursor.close()

def select_queries(cursor):
    selectpoblacionquery = "SELECT * FROM poblacion"
    selectsectoresquery = "SELECT * FROM sectores"
    
    cursor.execute(selectsectoresquery)
    data = cursor.fetchall()
    print("------Numero de sectores: ")
    print("-------- Sectores ---------- \n")
    print_cursor_result_sectores(data)
    
    cursor.execute(selectpoblacionquery)
    data = cursor.fetchall()
    print("------Numero de personas: ")
    print('\n')
    print("-------- Población ---------- \n")
    print_cursor_result_poblacion(data)

def drop_queries(cursor):
    dropsectoresquery = "DROP TABLE IF EXISTS sectores"
    droppoblacionquery = "DROP TABLE IF EXISTS poblacion"
    cursor.execute(droppoblacionquery)
    cursor.execute(dropsectoresquery)

def create_queries(cursor):
    createsectoresquery = """CREATE TABLE sectores (
                                  codS INTEGER,
                                  nombreS TEXT,
                                  porcentS INTEGER,
                                  ingresosS INTEGER,
                                  PRIMARY KEY (codS)
                                )"""
    createpoblacionquery = """CREATE TABLE poblacion (
                                  dni TEXT,
                                  nombre TEXT,
                                  apellido1 TEXT,
                                  apellido2 TEXT,
                                  fechanac DATE,
                                  direccion TEXT,
                                  cp TEXT,
                                  sexo TEXT,
                                  ingresos INTEGER,
                                  gastosFijos INTEGER,
                                  gastosAlim INTEGER,
                                  gastosRopa INTEGER,
                                  sector INTEGER,
                                  PRIMARY KEY (dni),
                                  FOREIGN KEY (sector) REFERENCES sectores (codS) ON DELETE SET NULL
                                );"""
    cursor.execute(createsectoresquery)
    cursor.execute(createpoblacionquery)

def insert_queries(connection, cursor):
    try:
        with open('insert_script.sql', 'r') as myscript:
            data = myscript.read()
        cursor.executescript(data)
        connection.commit()
    except:
        print('Error inserting data')
        connection.rollback()


def update_queries(connection, cursor):
    updatesectoresquery = """UPDATE sectores SET nombreS = 'Desconocido' WHERE codS = 4"""
    updatepoblacionquery = """UPDATE poblacion SET cp = 20251 WHERE dni = '111000111'"""

    print("Contenido de las tuplas antes de la ejecución: ")
    cursor.execute("SELECT * FROM sectores WHERE codS = 4")
    sector = cursor.fetchone()
    print_sectores_row(sector)

    cursor.execute("SELECT * FROM poblacion WHERE dni = '111000111'")
    poblacion = cursor.fetchone()
    print_poblacion_row(poblacion)
    print('\n')

    try:
        cursor.execute(updatepoblacionquery)
        cursor.execute(updatesectoresquery)
        connection.commit()
    except:
        print("Error updating")
        connection.rollback()

    print("Contenido de las tuplas después de la ejecución: ")
    cursor.execute("SELECT * FROM sectores WHERE codS = 4")
    sector = cursor.fetchone()
    print_sectores_row(sector)

    cursor.execute("SELECT * FROM poblacion WHERE dni = '111000111'")
    poblacion = cursor.fetchone()
    print_poblacion_row(poblacion)

def delete_queries(connection, cursor):
    deletepoblacionquery = "DELETE FROM poblacion WHERE dni = '111000111'"
    try:
        cursor.execute(deletepoblacionquery)
        connection.commit()
        print("Deleted poblacion row with dni 111000111")
    except:
        connection.rollback()
        print("Error deleting row")

def print_sectores_row(row):
    print("--------------------------------------------")
    print("Sector " + str(row[0]) + ", " + str(row[1]))
    print("Ingresos = " + str(row[3]))
    print("Porcentaje respecto al total -> " + str(row[2]))

def print_poblacion_row(row):
    print("--------------------------------------------")
    print("DNI " + str(row[0]) + " - " + str(row[1]) + " " + str(row[2]) + " " + str(row[3]))
    print("Fecha de nacimiento: " + str(row[4]))
    print("Dirección: " + str(row[5]))
    print("Codigo postal: " + str(row[6]))
    print("Sexo: " + str(row[7]))
    print("Ingresos: " + str(row[8]))
    print("Gastos fijos: " + str(row[9]))
    print("Gastos alimentación: " + str(row[10]))
    print("Gastos ropa: " + str(row[11]))
    print("Sector: " + str(row[12]))

def print_cursor_result_sectores(result):
    try:
        for row in result:
            print_sectores_row(row)
    except:
        print("Error: Unable to fetch data")
    print("--------------------------------------------")


def print_cursor_result_poblacion(result):
    try:
        for row in result:
            print_poblacion_row(row)
    except:
        print("Error: Unable to fetch data")
    print("--------------------------------------------")

In [18]:
connection = stablish_connection('pbd_sample')
cursor = create_cursor(connection)
select_queries(cursor)
delete_queries(connection, cursor)
drop_queries(cursor)
create_queries(cursor)
insert_queries(connection, cursor)
select_queries(cursor)
close_cursor(cursor)
close_connection(connection)

------Numero de sectores: 
-------- Sectores ---------- 

--------------------------------------------
Sector 1, Agricultura y pesca
Ingresos = 0
Porcentaje respecto al total -> 0
--------------------------------------------
Sector 2, Industria y energía
Ingresos = 0
Porcentaje respecto al total -> 0
--------------------------------------------
Sector 3, Servicios
Ingresos = 0
Porcentaje respecto al total -> 0
--------------------------------------------
Sector 4, Desconocido
Ingresos = 0
Porcentaje respecto al total -> 0
--------------------------------------------
------Numero de personas: 


-------- Población ---------- 

--------------------------------------------
DNI 123456789 - Carlos Gutiérrez Pérez
Fecha de nacimiento: 19971115
Dirección: Pz. Colón
Codigo postal: 06300
Sexo: H
Ingresos: 15000
Gastos fijos: 4000
Gastos alimentación: 4000
Gastos ropa: 3000
Sector: 1
--------------------------------------------
DNI 777888999 - Gerardo Martín Duque
Fecha de nacimiento: 19611212
D